In [ ]:
from enum import Enum
import json
from openai.types.chat.completion_create_params import ResponseFormat
from openai.types.chat import ChatCompletionMessageParam
from models.phone import PhoneModel
from service.phone import search, PhoneFilter
from uuid import uuid4
from models.user import UserRole
from repositories.user import create as create_user, CreateUserModel
from repositories.thread import create as create_thread, CreateThreadModel
from service.store_chatbot_v2 import gen_answer
from service.openai import  _client
import random
import math
from deepeval.test_case import LLMTestCase, ConversationalTestCase

from utils import EvaluateContext


phones = search(
    PhoneFilter()
)
phone = phones[3]

class Step(str, Enum):
    GREETING_AND_PROVIDE_NEED = "greeting and provide needs about the phone"
    SEARCH_PHONE_BASE_ON_THE_BRAND = "search phone base on the brand"
    SEARCH_PHONE_BASE_ON_THE_PRICE = "search phone base on the price"
    SELECT_ONE_PHONE_FROM_THE_LIST = "select one phone from the list"
    ASK_FOR_THE_DETAILS_OF_THE_SELECTED_PHONE = "ask for the details of the selected phone"
    PROVIDE_PHONE_NUMBER = "provide phone number"
    PROVIDE_EMAIL = "provide email"

class VietnameseUserSimulator:
    def __init__(self, phone: PhoneModel):
        user_info = self.generate_user_info()

        print(f"Generated user info: {user_info}")

        self.name = user_info["name"]
        self.age = user_info["age"]
        self.gender = user_info["gender"]
        self.phone_number = user_info["phone_number"]
        self.email = user_info["email"]

        # Calculate raw budget values
        raw_min_budget = min(phone.price * 0.9, phone.price - 1000000)
        if raw_min_budget < 0:
            raw_min_budget = 0
        raw_max_budget = max(phone.price * 1.1, 0, phone.price + 1000000)

        # Round to millions
        self.min_budget = math.floor(raw_min_budget / 1000000) * 1000000
        self.max_budget = math.ceil(raw_max_budget / 1000000) * 1000000

        print(f"Original price: {phone.price:,} VND")
        print(f"Rounded budget range: {self.min_budget:,} - {self.max_budget:,} VND")

        self.basic_phone_info = (
            phone.to_text(include_key_selling_points=True)
            + f"- Brand: {phone._get_brand_name()}"
        )
        self.full_phone_info = phone.to_text(True, True, True, True)
        self.response_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "Response",
                "description": "Response from you to the latest user message.",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "response_message": {
                            "type": "string",
                            "description": "Response message for the latest user message. It can be a question or a statement. It should be concise and in Vietnamese.",
                        },
                        "current_step_for_step": {
                            "type": "string",
                            "enum": [step.value for step in Step],
                            "description": "Current step of the response message. It should be one of the steps in the Step enum and match the current step of the conversation.",
                        },
                    },
                    "additionalProperties": False,
                    "required": ["current_step_for_step", "response_message"],
                },
            },
        }
        self.step_history: list[Step] = []
        self.conversation_history: list[ChatCompletionMessageParam] = []
        self.user = create_user(
            CreateUserModel(
                user_name=str(uuid4()), role=UserRole.chainlit_user, gender=self.gender
            )
        )
        self.thread = create_thread(
            CreateThreadModel(id=uuid4(), user_id=self.user.id, name=self.name)
        )
        self.llm_test_cases: list[LLMTestCase] = [] 

    def generate_user_info(self) -> dict:
        """Generate realistic Vietnamese user information using OpenAI API"""
        user_info_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "VietnameseUserInfo",
                "description": "Information about a Vietnamese user",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "Full Vietnamese name with proper accents",
                        },
                        "age": {
                            "type": "integer",
                            "description": "Age between 18 and 65",
                        },
                        "gender": {
                            "type": "string",
                            "enum": ["male", "female"],
                            "description": "Gender of the user",
                        },
                        "phone_number": {
                            "type": "string",
                            "description": "Valid Vietnamese phone number format, starting with 0",
                        },
                        "email": {
                            "type": "string",
                            "description": "Realistic email address that relates to the person's name",
                        },
                    },
                    "additionalProperties": False,
                    "required": ["name", "age", "gender", "phone_number", "email"],
                },
            },
        }

        messages: list[ChatCompletionMessageParam] = [
            {
                "role": "system",
                "content": "You are a helper that generates realistic Vietnamese user profiles. Generate a random Vietnamese user profile with a name (including proper Vietnamese accents), age (between 18-65), gender, phone number (starting with 0, total 10 digits), and an email address that reflects the person's name."
            },
            {
                "role": "user",
                "content": "Generate a random Vietnamese person's information."
            }
        ]

        try:
            response = _client.chat.completions.create(
                messages=messages,
                model="gpt-4.1-mini",
                temperature=0.8,
                response_format=user_info_format,
                timeout=10
            )

            user_info = json.loads(response.choices[0].message.content or "{}")
            return user_info
        except Exception as e:
            print(f"Error generating user info: {e}")
            # Fallback to default values
            return {
                "name": f"Nguyễn Văn {random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])}",
                "age": random.randint(18, 65),
                "gender": random.choice(["male", "female"]),
                "phone_number": f"09{random.randint(10000000, 99999999)}",
                "email": f"nguyen{random.choice(['a', 'b', 'c', 'd', 'e', 'f', 'g'])}@gmail.com"
            }

    def get_system_prompt(self) -> list[ChatCompletionMessageParam]:
        role = (
            "# ROLE\n"
            "You are a Vietnamese virtual user playing the role of a customer searching for a new phone. You are chatting with an online customer service agent.\n"
        )
        profile = (
            f"## PROFILE\n"
            f"- Name: {self.name}\n"
            f"- Age: {self.age}\n"
            f"- Gender: {self.gender}\n"
            f"- Phone number: {self.phone_number}\n"
            f"- Email: {self.email}\n"
            f"- Min budget: {self.min_budget}\n"
            f"- Max budget: {self.max_budget}\n"
        )

        latest_step_in_past = self.step_history[-1] if self.step_history else None

        phone_looking_for = (
            (f"## INFORMATION ABOUT PHONE LOOKING FOR\n" f"{self.basic_phone_info}\n")
            if latest_step_in_past
            in [
                Step.GREETING_AND_PROVIDE_NEED,
                Step.SEARCH_PHONE_BASE_ON_THE_BRAND,
                Step.SEARCH_PHONE_BASE_ON_THE_PRICE,
                Step.SELECT_ONE_PHONE_FROM_THE_LIST,
            ]
            else (
                f"## INFORMATION ABOUT PHONE LOOKING FOR\n" f"{self.full_phone_info}\n"
            )
        )

        step_descriptions = (
            "## STEP DESCRIPTIONS\n"
            f"1. **{Step.GREETING_AND_PROVIDE_NEED.value}**: Greet the customer support agent and provide your needs about the phone. Example: 'Mình cần tư vấn điện thoại', 'Hello', 'Mình cần mua điện thoại tầm {self.min_budget} đến {self.max_budget} VNĐ', 'Xin chào', 'Tôi cần một chiếc điện thoại mới'.\n"
            f"2. **{Step.SEARCH_PHONE_BASE_ON_THE_BRAND.value}**: Search for a phone based on the brand. Example: 'Tìm điện thoại {phone._get_brand_name()}', 'Tìm điện thoại thương hiệu {phone._get_brand_name()}', '{phone._get_brand_name()}', 'hãng {phone._get_brand_name()}'.\n"
            f"3. **{Step.SEARCH_PHONE_BASE_ON_THE_PRICE.value}**: Search for a phone based on the price. Example: 'Tìm điện thoại dưới {self.min_budget} VNĐ', 'Tìm điện thoại trên {self.max_budget} VNĐ', 'Tìm điện thoại giá {self.min_budget} đến {self.max_budget} VNĐ'.\n"
            f"4. **{Step.SELECT_ONE_PHONE_FROM_THE_LIST.value}**: Select one phone from the suggested list in past. Example: 'Chọn điện thoại {phone.name} trong danh sách', 'Chọn điện thoại {phone.name}', 'cái đầu', 'mẫu số 2'.\n"
            f"5. **{Step.ASK_FOR_THE_DETAILS_OF_THE_SELECTED_PHONE.value}**: Ask for the details of the selected phone by analyzing the information in the <INFORMATION ABOUT PHONE LOOKING FOR> section. Extract key specifications, features, and selling points from this section, and formulate natural, relevant questions about these aspects. Generate diverse questions that someone would genuinely ask when considering purchasing this specific phone model. Vary your questions between technical specifications, features, promotions, colors, accessories, user experience, and purchase conditions.\n"
            f"6. **{Step.PROVIDE_PHONE_NUMBER.value}**: Provide your phone number when you need further consultation or are ready to purchase. Example: 'Số điện thoại của mình là {self.phone_number}'.\n"
            f"7. **{Step.PROVIDE_EMAIL.value}**: Provide your email. Example: 'Email của mình là {self.email}'.\n"
        )

        if latest_step_in_past:
            count = 0
            for step in reversed(self.step_history):
                if step == latest_step_in_past:
                    count += 1
                else:
                    break

            # Add special guidance for Step 5 to help generate diverse questions
            if latest_step_in_past == Step.ASK_FOR_THE_DETAILS_OF_THE_SELECTED_PHONE:
                asked_topics = self.extract_asked_topics()
                if asked_topics:
                    suggested_topics = self.suggest_new_topics(asked_topics)
                    step_descriptions += (
                        "\n## QUESTION HISTORY AND SUGGESTIONS\n"
                        f"You have already asked about: {', '.join(asked_topics)}.\n"
                        f"Consider asking about new topics such as: {', '.join(suggested_topics)}.\n"
                    )

            step_descriptions += (
                "\n## LATEST STEP IN PAST\n"
                f"Latest step in past: {latest_step_in_past.value}\n"
                f"Stay at step {latest_step_in_past.value} for {count} turns.\n"
            )

        task = (
            "## TASK\n"
            "Generate a response message for the latest user message based on the current step of the conversation. It's like talking to a real customer service agent."
        )

        guidelines = (
            "## GUIDELINES\n"
            "1. The response message should be in Vietnamese.\n"
            "2. When starting the conversation, greet the customer support agent and provide your needs about the phone. (Step 1)\n"
            "3. If the user asks for the type of product that you are looking for, provide the type of product that you are looking for is a phone.\n"
            f"4. If the user asks for the brand of the phone that you are looking for, provide the brand of the phone that you are looking for is {phone._get_brand_name()} (Step 2).\n"
            f"5. If the user asks for the price of the phone that you are looking for, provide the price of the phone that you are looking for is between {self.min_budget} and {self.max_budget} (Step 3).\n"
            f"6. If the user provides a list of phones and has a phone that you are looking for ({phone.name}), select that phone from the list (Step 4).\n"
            "7. If the user provides the details of the selected phone and asks your contact information, ask for the details of the selected phone (Step 5).\n"
            "8. If the user provides the details of the selected phone and the latest step in past is Step 5, provide your phone number or email (Step 6 or Step 7).\n"
            "\n## NOTE:\n"
            "- Imagine you are a real customer who has just interacted with a business. Your response should sound natural and authentic.\n"
            "- You need to stay at the Step 5 minimum 3 turns and maximum 5 turns before moving to Step 6 or Step 7.\n"
            f"- If you can't find the phone ({phone.name}) in the list of phones suggested by the customer service agent, you can ask for other phones (e.g., 'Có mẫu nào khác không?'). "
            "If still unavailable, then provide your contact information (Step 6 or Step 7).\n"
        )

        return [
            {"role": "system", "content": role + '\n' + profile},
            {"role": "system", "content": phone_looking_for},
            {"role": "system", "content": step_descriptions},
            {"role": "system", "content": task},
            {"role": "system", "content": guidelines},
        ]

    def extract_asked_topics(self) -> list[str]:
        """Extract topics that the user has already asked about using OpenAI API"""
        # If no conversation yet, return empty list
        if len(self.conversation_history) < 2:
            return []

        # Create a format for the response
        topic_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "AskedTopics",
                "description": "Topics that the user has already asked about in the conversation",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "topics": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of topics that have been asked about in the conversation",
                        }
                    },
                    "additionalProperties": False,
                    "required": ["topics"],
                },
            },
        }

        # Get only the last few messages to avoid token limits
        recent_messages = self.conversation_history[-10:] if len(self.conversation_history) > 10 else self.conversation_history

        # Create the prompt for OpenAI
        messages = [
            {
                "role": "system",
                "content": "You are an assistant that analyzes conversation history to identify what topics a customer has already asked about regarding a phone. Extract key topics the customer has asked about such as battery life, camera quality, screen size, price, etc."
            },
            {
                "role": "user",
                "content": f"Here is a conversation between a customer and a phone store assistant. Identify what specific topics about the phone the customer has already asked about in these messages:\n\n" + 
                           "\n".join([f"{'Customer' if msg['role'] == 'assistant' else 'Assistant'}: {msg.get('content', '')}" for msg in recent_messages])
            }
        ]

        try:
            response = _client.chat.completions.create(
                messages=messages,
                model="gpt-4.1-mini",
                temperature=0.3,
                response_format=topic_format,
                timeout=10
            )

            result = json.loads(response.choices[0].message.content or "{}")
            topics = result.get("topics", [])
            return topics[:10]  # Limit to 10 topics
        except Exception as e:
            print(f"Error extracting topics: {e}")
            # Fallback to basic topic extraction
            return ["general phone information"]

    def suggest_new_topics(self, asked_topics: list[str]) -> list[str]:
        """Suggest topics that haven't been asked about yet using OpenAI API"""
        # Setup the response format
        topic_format: ResponseFormat = {
            "type": "json_schema",
            "json_schema": {
                "name": "SuggestedTopics",
                "description": "Topics that could be asked about the phone",
                "strict": True,
                "schema": {
                    "type": "object",
                    "properties": {
                        "suggested_topics": {
                            "type": "array",
                            "items": {"type": "string"},
                            "description": "List of suggested topics about the phone that haven't been asked yet",
                        }
                    },
                    "additionalProperties": False,
                    "required": ["suggested_topics"],
                },
            },
        }

        # Create the prompt for OpenAI
        messages = [
            {
                "role": "system",
                "content": "You are an assistant that suggests relevant topics a customer could ask about a phone. Given the phone details and topics already asked, suggest new topics that would be helpful for making a purchase decision."
            },
            {
                "role": "user",
                "content": f"Phone information:\n{self.full_phone_info}\n\nTopics already asked about:\n{', '.join(asked_topics)}\n\nSuggest 5 other relevant topics the customer could ask about this phone that haven't been covered yet."
            }
        ]

        try:
            response = _client.chat.completions.create(
                messages=messages,
                model="gpt-4.1-mini",
                temperature=0.7,
                response_format=topic_format,
                timeout=10
            )

            result = json.loads(response.choices[0].message.content or "{}")
            suggested_topics = result.get("suggested_topics", [])
            return suggested_topics[:5]  # Limit to 5 topics
        except Exception as e:
            print(f"Error suggesting topics: {e}")
            # Fallback to some generic topics
            return ["special features", "warranty policy", "accessories", "user experience", "purchase options"]

    def get_next_user_message(self) -> str:

        messages: list[ChatCompletionMessageParam] = [
            *self.get_system_prompt(),
            *self.conversation_history,
        ]
        response = _client.chat.completions.create(
            messages=messages,
            model="gpt-4.1-mini",
            temperature=0.7,
            timeout=30,
            response_format=self.response_format,
        )
        response_message = response.choices[0].message.content
        parsed_information = json.loads(response_message or "{}")
        print(f"Response: {parsed_information} in turn {len(self.conversation_history) + 1}")
        current_step_for_step = parsed_information.get("current_step_for_step")
        response_message = parsed_information.get("response_message")
        if current_step_for_step and response_message:
            self.step_history.append(Step(current_step_for_step))
            self.conversation_history.append(
                {
                    "role": "assistant",
                    "content": response_message,
                }
            )
            
            evaluate_context = EvaluateContext()
            assistant_response = gen_answer(user_id=self.user.id, thread_id=self.thread.id, history=self.get_reversed_role_in_conversation_history(), evaluate_context=evaluate_context)

            self.conversation_history.append(
                {
                    "role": "user",
                    "content": assistant_response,
                }
            )
            self.llm_test_cases.append(
                LLMTestCase(
                    input=response_message,
                    actual_output= assistant_response,
                    retrieval_context= evaluate_context.knowledge,
                    additional_metadata=evaluate_context.model_dump()
                )
            )
            return response_message
        else:
            raise ValueError("Invalid response format")

    def get_reversed_role_in_conversation_history(self) -> list[ChatCompletionMessageParam]:
        reversed_history = []
        for message in self.conversation_history:
            # Only include messages that have a 'content' key
            if "content" in message and "role" in message:
                reversed_history.append({
                    "role": "user" if message["role"] == "assistant" else "assistant",
                    "content": message["content"],
                })
        return reversed_history

    def simulate_conversation(self, max_turns: int = 20):
        if not self.conversation_history:
            self.conversation_history.append(
                {
                    "role": "user",
                    "content": "Xin chào, bạn cần hỗ trợ gì ạ?",
                }
            )
        for _ in range(max_turns):
            self.get_next_user_message()
            if self.step_history[-1] == Step.PROVIDE_EMAIL or self.step_history[-1] == Step.PROVIDE_PHONE_NUMBER:
                print(f"User {self.name} has provided their contact information.")
                break
        for message in self.conversation_history:
            print(f"{message['role']}: {message.get('content')}")
        for step in self.step_history:
            print(f"Step: {step.value}")

In [9]:
simulate_user = VietnameseUserSimulator(phone)
simulate_user.simulate_conversation()

2025-05-24 17:41:52 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Generated user info: {'name': 'Nguyễn Thị Hương', 'age': 29, 'gender': 'female', 'phone_number': '0912345678', 'email': 'nguyentihuong29@gmail.com'}
Original price: 22,990,000 VND
Rounded budget range: 20,000,000 - 26,000,000 VND
2025-05-24 17:41:53 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Response: {'response_message': 'Xin chào, mình cần tư vấn mua điện thoại mới với mức giá từ 20 triệu đến 26 triệu đồng.', 'current_step_for_step': 'greeting and provide needs about the phone'} in turn 2
2025-05-24 17:41:55 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
User request: {'user_demand': <ProductType.UNDETERMINED: 'undetermined'>, 'user_info': {'phone_number': None, 'email': None}}
Detect demand response: type='finished' content='The user request has been successfully processed.' instructions=[] UserIntent(is_user_

In [10]:
from deepeval.test_case import LLMTestCase, ConversationalTestCase